In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from fuzzywuzzy import fuzz
from tqdm import tqdm

warnings.filterwarnings("ignore")
%matplotlib inline

In [51]:
df = pd.read_csv('../transfers1.2.csv')
df.head()

,ID,to_club_name,to_league_name,player_name,age,position,from_club_involved_name,fee,transfer_movement,fee_cleaned,year,season,grouping_positions_1,grouping_positions_2
0,1,Paris Saint-Germain,Ligue 1,Neymar,25,Left Winger,FC Barcelona,£199.80m,in,199.8,2017,2017/2018,LW,W
1,2,FC Barcelona,Primera Division,Philippe Coutinho,25,Left Winger,Liverpool,£130.50m,in,130.5,2017,2017/2018,LW,W
2,3,Paris Saint-Germain,Ligue 1,Kylian Mbappé,19,Centre Forward,Monaco,£121.50m,in,121.5,2018,2018/2019,CF,F
3,4,Atlético Madrid,Primera Division,João Félix,19,Second Striker,Benfica,£113.40m,in,113.4,2019,2019/2020,CF,F
4,5,FC Barcelona,Primera Division,Ousmane Dembélé,20,Left Winger,Bor. Dortmund,£112.50m,in,112.5,2017,2017/2018,LW,W


In [52]:
df.to_league_name.value_counts()

Serie A             1618
Premier League      1202
1 Bundesliga         982
Ligue 1              837
Primera Division     809
Premier Liga         538
Liga Nos             378
Eredivisie           370
Name: to_league_name, dtype: int64

In [53]:
# writing a loop to classify the clubs

PL = []
la_liga = []
serie_a = []
bundesliga = []
ligue_1 = []
premier_liga = []
liga_nos = []
eredivisie = []

for i in range(df.shape[0]):
    if(df.to_league_name[i] == 'Premier League'):
        PL.append(df.to_club_name[i])
    elif(df.to_league_name[i] == 'Serie A'):
        serie_a.append(df.to_club_name[i])
    elif(df.to_league_name[i] == 'Primera Division'):
        la_liga.append(df.to_club_name[i])
    elif(df.to_league_name[i] == '1 Bundesliga'):
        bundesliga.append(df.to_club_name[i])
    elif(df.to_league_name[i] == 'Ligue 1'):
        ligue_1.append(df.to_club_name[i])  
    elif(df.to_league_name[i] == 'Premier Liga'):
        premier_liga.append(df.to_club_name[i])
    elif(df.to_league_name[i] == 'Liga Nos'):
        liga_nos.append(df.to_club_name[i])    
    else:
        eredivisie.append(df.to_club_name[i])
        
#making all lists unique

PL = set(PL)
PL = list(PL)

la_liga = set(la_liga)
la_liga = list(la_liga)

serie_a = set(serie_a)
serie_a = list(serie_a)

bundesliga = set(bundesliga)
bundesliga = list(bundesliga)

ligue_1 = set(ligue_1)
ligue_1 = list(ligue_1)

premier_liga = set(premier_liga)
premier_liga = list(premier_liga)

liga_nos = set(liga_nos)
liga_nos = list(liga_nos)

eredivisie = set(eredivisie)
eredivisie = list(eredivisie)

df['normalized_from_club_name'] = 0

I tried to run all the following loops in one single loop, but it was too heavy computationally, hence I broke it down for each league. 

In [54]:
# Premier League

from_league = []
for i in tqdm(range(df.shape[0])):

    ratio_pl = []
    for j in range(len(PL)):
        ratio_pl.append(fuzz.partial_ratio(df.from_club_involved_name[i].lower(), PL[j].lower()))
        ratio = max(ratio_pl)
        
    if(ratio>80):
        val = ratio_pl.index(ratio)
        df['normalized_from_club_name'][i] = PL[val]
        from_league.append('Premier League')
    else:
        from_league.append('Other')

df['from_league'] = from_league   

100%|█████████████████████████████████████████████████████████████████████████████| 6734/6734 [00:08<00:00, 828.54it/s]


In [55]:
# Serie A

for i in tqdm(range(df.shape[0])):
    
    if(df['from_league'][i] ==  'Other'):
        
        ratio_sa = []
        for k in range(len(serie_a)):
            ratio_sa.append(fuzz.partial_ratio(df.from_club_involved_name[i].lower(), serie_a[k].lower()))
           # print(ratio_sa)
            ratio = max(ratio_sa)
            
        if(ratio>80):
            val = ratio_sa.index(ratio)
            df['normalized_from_club_name'][i] = serie_a[val]
            df['from_league'][i] = 'Serie A'
        else:
            from_league.append('Other')
    else:
        None

100%|█████████████████████████████████████████████████████████████████████████████| 6734/6734 [00:07<00:00, 907.14it/s]


In [56]:
# Bundesliga

for i in tqdm(range(df.shape[0])):
    
    if(df['from_league'][i] ==  'Other'):
        
        ratio_bl = []
        for m in range(len(bundesliga)):
            ratio_bl.append(fuzz.partial_ratio(df.from_club_involved_name[i].lower(), bundesliga[m].lower()))
            ratio = max(ratio_bl)
            
        if(ratio>80):
            val = ratio_bl.index(ratio)
            df['normalized_from_club_name'][i] = bundesliga[val]
            df['from_league'][i]= '1 Bundesliga'
        else:
            from_league.append('Other')
    else:
        None


100%|████████████████████████████████████████████████████████████████████████████| 6734/6734 [00:04<00:00, 1396.41it/s]


In [57]:
# Ligue 1

for i in tqdm(range(df.shape[0])):
    
    if(df['from_league'][i] ==  'Other'):
        
        ratio_l = []
        for m in range(len(ligue_1)):
            ratio_l.append(fuzz.partial_ratio(df.from_club_involved_name[i].lower(), ligue_1[m].lower()))
            ratio = max(ratio_l)
            
        if(ratio>80):
            val = ratio_l.index(ratio)
            df['normalized_from_club_name'][i] = ligue_1[val]
            df['from_league'][i]= 'Ligue 1'
        else:
            from_league.append('Other')
    else:
        None


100%|████████████████████████████████████████████████████████████████████████████| 6734/6734 [00:05<00:00, 1198.20it/s]


In [58]:
# Premier Liga

for i in tqdm(range(df.shape[0])):
    
    if(df['from_league'][i] ==  'Other'):
        
        ratio_l = []
        for m in range(len(premier_liga)):
            ratio_l.append(fuzz.partial_ratio(df.from_club_involved_name[i].lower(), premier_liga[m].lower()))
            ratio = max(ratio_l)
            
        if(ratio>80):
            val = ratio_l.index(ratio)
            df['normalized_from_club_name'][i] = premier_liga[val]
            df['from_league'][i]= 'Premier Liga'
        else:
            from_league.append('Other')
    else:
        None


100%|████████████████████████████████████████████████████████████████████████████| 6734/6734 [00:03<00:00, 1949.05it/s]


In [59]:
# Liga Nos

for i in tqdm(range(df.shape[0])):
    
    if(df['from_league'][i] ==  'Other'):
        
        ratio_l = []
        for m in range(len(liga_nos)):
            ratio_l.append(fuzz.partial_ratio(df.from_club_involved_name[i].lower(), liga_nos[m].lower()))
            ratio = max(ratio_l)
            
        if(ratio>80):
            val = ratio_l.index(ratio)
            df['normalized_from_club_name'][i] = liga_nos[val]
            df['from_league'][i]= 'Liga Nos'
        else:
            from_league.append('Other')
    else:
        None


100%|████████████████████████████████████████████████████████████████████████████| 6734/6734 [00:03<00:00, 2129.16it/s]


In [60]:
# Eredivisie

for i in tqdm(range(df.shape[0])):
    
    if(df['from_league'][i] ==  'Other'):
        
        ratio_l = []
        for m in range(len(eredivisie)):
            ratio_l.append(fuzz.partial_ratio(df.from_club_involved_name[i].lower(), eredivisie[m].lower()))
            ratio = max(ratio_l)
            
        if(ratio>80):
            val = ratio_l.index(ratio)
            df['normalized_from_club_name'][i] = eredivisie[val]
            df['from_league'][i]= 'Eredivisie'
        else:
            from_league.append('Other')
    else:
        None


100%|████████████████████████████████████████████████████████████████████████████| 6734/6734 [00:02<00:00, 2350.16it/s]


In [61]:
# Primera Division

for i in tqdm(range(df.shape[0])):
    
    if(df['from_league'][i] ==  'Other'):
        
        ratio_l = []
        for m in range(len(la_liga)):
            ratio_l.append(fuzz.partial_ratio(df.from_club_involved_name[i].lower(), la_liga[m].lower()))
            ratio = max(ratio_l)
            
        if(ratio>80):
            val = ratio_l.index(ratio)
            df['normalized_from_club_name'][i] = la_liga[val]
            df['from_league'][i]= 'Primera Division'
        else:
            from_league.append('Other')
    else:
        None


100%|████████████████████████████████████████████████████████████████████████████| 6734/6734 [00:03<00:00, 1796.17it/s]


In [62]:
df['from_league'].value_counts()

Other               2712
Serie A             1033
Ligue 1              667
Premier League       607
Primera Division     538
1 Bundesliga         406
Eredivisie           323
Liga Nos             275
Premier Liga         173
Name: from_league, dtype: int64

In [63]:
df.head()

,ID,to_club_name,to_league_name,player_name,age,position,from_club_involved_name,fee,transfer_movement,fee_cleaned,year,season,grouping_positions_1,grouping_positions_2,normalized_from_club_name,from_league
0,1,Paris Saint-Germain,Ligue 1,Neymar,25,Left Winger,FC Barcelona,£199.80m,in,199.8,2017,2017/2018,LW,W,FC Barcelona,Primera Division
1,2,FC Barcelona,Primera Division,Philippe Coutinho,25,Left Winger,Liverpool,£130.50m,in,130.5,2017,2017/2018,LW,W,Liverpool FC,Premier League
2,3,Paris Saint-Germain,Ligue 1,Kylian Mbappé,19,Centre Forward,Monaco,£121.50m,in,121.5,2018,2018/2019,CF,F,AS Monaco,Ligue 1
3,4,Atlético Madrid,Primera Division,João Félix,19,Second Striker,Benfica,£113.40m,in,113.4,2019,2019/2020,CF,F,SL Benfica,Liga Nos
4,5,FC Barcelona,Primera Division,Ousmane Dembélé,20,Left Winger,Bor. Dortmund,£112.50m,in,112.5,2017,2017/2018,LW,W,0,Other


In [64]:
df.to_csv('../transfers1.3.csv')